# Building your first Agent Systems with Databricks

LLMs are powerful to answer general knowledge, but don't have any information on your own business.

Databricks makes it easy to develop custom tools which can be called by your LLM, creating a complete agent with reasoning capability.


### Build Simple UC Tools
In this notebook, we'll register the following to Unity Catalogs:
- **SQL Functions**: Create queries that access customer information, order and subscriptions.
- **Simple Python Function**: Create and register a Python function doing math operations to overcome some common limitations of language models.

>

In [0]:
%pip install databricks-agents mlflow>=3.1.0 databricks-sdk==0.55.0 unitycatalog-ai[databricks]
# Restart to load the packages into the Python environment
dbutils.library.restartPython()

In [0]:
%run ../_config/config_unity_catalog

Generate synthetic data we're going to use with our agents.

# I- Create the functions

## 1. check_order_status(order_id)

Returns current order status and tracking information

    Returns: order status, tracking_number, estimated_delivery
    Joins orders table with order_items for complete order details


In [0]:
%sql
CREATE OR REPLACE FUNCTION check_order_status(order_id INT COMMENT 'order_id of the command you want to get status')
RETURNS TABLE (
    status STRING,
    tracking_number STRING,
    total_amount DOUBLE,
    estimated_delivery STRING
)
COMMENT 'Returns current order status,  tracking information and estimated delivery'
RETURN (
  SELECT 
    status,
    tracking_number,
    total_amount,
    CASE 
      WHEN status = 'delivered' THEN 'Delivered'
      WHEN status = 'shipped' THEN 'In Transit - 1-3 days'
      WHEN status = 'processing' THEN 'Processing - ships in 1-2 days'
      WHEN status = 'pending' THEN 'Pending - ships within 24 hours'
      ELSE 'Status Unknown'
    END as estimated_delivery
  FROM orders_feature
  WHERE order_id = order_id
  limit 1
);


Test the function

In [0]:
%sql SELECT * FROM check_order_status(1003);


## 2. calculate_shipping(customer_id, total_weight_kg, destination_country, destination_region)
Python function to calculate shipping cost
- Returns: shipping cost, estimated delivery days, available options
- Considers customer tier (Gold gets free express shipping)
- Looks up shipping_zones table for rates

In [0]:
# -----------------------
# TOOL 2: evaluate shipping cost
# -----------------------
def calculate_shipping(
  customer_id: int,
  total_weight_kg: float,
  destination_country: str,
  destination_region: str
) -> dict[str, float]:
    """
    Evaluates the shipping cost.

    Args:
        customer_id : id of a customer.
        total_weight_kg : total weight of the order in kg.
        destination_country : destination country.
        destination_region : destination region.

    Returns:
        a dict with the following keys:
        - standard_cost: float: The standard shipping cost.
        - expedited_cost: float: The expedited shipping cost.
        - express_cost: float: The express shipping cost.
        - estimated_days: str: The estimated number ofdays for delivery.

    """
    # Get customer tier for benefits
    customer_tier = spark.sql(f"SELECT tier FROM {catalog}.{schema}.customers_feature WHERE customer_id = {customer_id}").collect()[0]['tier']
    
    # Get base shipping rate for zone
    zone_info = spark.sql(f"""
        SELECT base_rate, rate_per_kg, estimated_days 
        FROM {catalog}.{schema}.shipping_zones_feature 
        WHERE country = '{destination_country}' AND region = '{destination_region}'
    """).collect()[0]
    
    base_rate = zone_info['base_rate']
    rate_per_kg = zone_info['rate_per_kg']
    estimated_days = zone_info['estimated_days']
    
    # Calculate standard shipping
    standard_cost = base_rate + (total_weight_kg * rate_per_kg)
    
    # Apply tier benefits
    if customer_tier == 'Gold':
        expedited_cost = 0  # Free expedited for Gold
        express_cost = 15.99  # Discounted express
    elif customer_tier == 'Silver':
        expedited_cost = 10.99  # Discounted expedited
        express_cost = 24.99
    else:  # Bronze
        expedited_cost = 15.99
        express_cost = 29.99
    
    return {
        'standard_cost': round(standard_cost, 2),
        'expedited_cost': expedited_cost,
        'express_cost': express_cost,
        'estimated_days': estimated_days
    }


## Test the python function
print(calculate_shipping(9, 22, 'USA', 'West Coast') )

## 3. check_product_stock(product_id)

- Returns: stock_quantity, warehouse_location, availability status
- Alerts if stock is low (< 10 units)

In [0]:
%sql


-- Function 2: Check Product Stock
CREATE or REPLACE FUNCTION check_product_stock(product_id INT COMMENT 'product_id used to retrive stock')
RETURNS TABLE (
    stock_quantity INT,
    warehouse_location STRING,
    availability STRING
)
COMMENT 'Returns stock quantity, warehouse location and availability.'
RETURN (
  SELECT 
    stock_quantity,
    warehouse_location,
    CASE
      WHEN stock_quantity > 50 THEN 'In Stock - Ships Today'
      WHEN stock_quantity BETWEEN 10 AND 50 THEN 'Limited Stock Available'
      WHEN stock_quantity BETWEEN 1 AND 9 THEN 'Low Stock - Order ASAP'
      ELSE 'Out of Stock'
    END as availability
  FROM products_feature
  WHERE product_id = product_id
)

In [0]:
%skip
%sql
SELECT *
FROM check_product_stock(1)
order by stock_quantity desc
limit 2;


## 4. get_customer_orders(customer_id)
- Returns: order history with status, dates, totals
- Joins orders with order_items and products for full details

In [0]:
%sql


CREATE OR REPLACE FUNCTION get_customer_orders(
    customer_id INT COMMENT 'customer_id is used to retrieve orders'
)
RETURNS TABLE (
    order_id INT,
    order_date DATE,
    status STRING,
    total_amount DOUBLE,
    order_items ARRAY<STRUCT<product_id INT, name STRING, price DOUBLE>>
)
COMMENT 'Returns customers orders, order date, status, total amount, and products details.'
RETURN (
  SELECT 
    of.order_id AS order_id,
    of.order_date AS order_date,
    of.status AS status,
    of.total_amount AS total_amount,
    array_agg(struct(pf.product_id, pf.name, pf.price)) AS order_items
  FROM orders_feature AS of
  LEFT JOIN order_items_feature AS oif
    ON of.order_id = oif.order_id
  LEFT JOIN products_feature AS pf
    ON pf.product_id = oif.product_id
  WHERE of.customer_id = customer_id
  GROUP BY
    of.order_id,
    of.order_date,
    of.status,
    of.total_amount
)

In [0]:
%skip
%sql
SELECT *
FROM get_customer_orders(9)
order by order_date desc
limit 3;

## 5. check_return_eligibility(order_id)

- Returns: eligible (yes/no), reason, days_remaining
- Checks order_date against current date
- Considers product category and order status
- Returns number of days left in return window

In [0]:
%sql

CREATE OR REPLACE FUNCTION check_return_eligibility(
    order_id INT COMMENT 'order_id is used to retrieve orders'
)
RETURNS TABLE (
    eligible BOOLEAN,
    reason STRING,
    days_remaining INT,
    name STRING,
    category STRING
)
COMMENT 'Returns customers orders, order date, status, total amount, and products details.'
RETURN (
  SELECT 
    CASE 
      WHEN of.status = 'delivered' AND DATEDIFF(CURRENT_DATE(), of.order_date) <= 30 THEN TRUE
      ELSE FALSE
    END as eligible,
    CASE
      WHEN of.status != 'delivered' THEN 'Order must be delivered to process return'
      WHEN DATEDIFF(CURRENT_DATE(), of.order_date) > 30 THEN 'Return window expired (30 days from delivery)'
      WHEN of.status = 'cancelled' THEN 'Cannot return cancelled order'
      ELSE 'Eligible for return'
    END as reason,
    GREATEST(0, 30 - DATEDIFF(CURRENT_DATE(), of.order_date)) as days_remaining,
    pf.name as name,
    pf.category as category

  FROM orders_feature AS of
  LEFT JOIN order_items_feature AS oif
    ON of.order_id = oif.order_id
  LEFT JOIN products_feature AS pf
  WHERE of.order_id = check_return_eligibility.order_id
)

In [0]:
%skip
%sql
SELECT *
FROM check_return_eligibility(1001);

## 6. get_product_details(product_id)

- Returns: full product information including price, specs, stock
- Used for product recommendations and comparisons

In [0]:
%sql

CREATE OR REPLACE FUNCTION get_product_details(
  product_name STRING COMMENT 'product_name is used to retrieve full infos about a product'
)
RETURNS TABLE (
    product_id INT,
    name STRING,
    price DOUBLE,
    category STRING,
    brand STRING,
    weight_kg DOUBLE,
    stock_quantity INT,
    warehouse_location STRING,
    distance DOUBLE
)
COMMENT 'Returns product details by a search by levenshein algorithm based on the product name given by the customer: product_id, name, category, brand, price, weight_kg, sotck_quantity, warehouse_location.'
RETURN (
  SELECT 
    product_id,
    name,
    price,
    category,
    brand,
    weight_kg,
    stock_quantity,
    warehouse_location,
    LEVENSHTEIN(LOWER(name), LOWER(product_name)) AS distance
  from products_feature as pf
  ORDER BY distance ASC
  LIMIT 1
);


test the function

In [0]:
%skip
%sql
SELECT *
FROM get_product_details("UltraView 4K monitor");

## 7. get_customer_tier_benefits(customer_id)

- Returns: tier level, benefits, lifetime_value, total_orders
- Shows eligibility for free shipping, discounts, priority support

In [0]:
%sql

CREATE OR REPLACE FUNCTION get_customer_tier_benefits(
  email STRING 
)
RETURNS TABLE (
    tier_level STRING,
    lifetime_value DOUBLE,
    total_orders INT
)
COMMENT 'Returns customer tier level,  lifetime_value, total_orders knowing customer email'
RETURN (
  SELECT 
    tier as tier_level,
    lifetime_value,
    total_orders
  from customers_feature as cf
  WHERE cf.email= get_customer_tier_benefits.email

)

Test the function.


In [0]:
%skip
%sql
SELECT *
FROM get_customer_tier_benefits("john.smith@email.com");

# II. Register python functions to UC

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

# this will deploy the tool to UC, automatically setting the metadata in UC based on the tool's docstring & typing hints
python_tool_uc_info = client.create_python_function(func=calculate_shipping, catalog=catalog, schema=schema, replace=True)

# the tool will deploy to a function in UC called `{catalog}.{schema}.{func}` where {func} is the name of the function
# Print the deployed Unity Catalog function name
print(f"Deployed Unity Catalog function name: {python_tool_uc_info.full_name}")
# Create HTML link to created functions
displayHTML(f'<a href="/explore/data/functions/{catalog}/{schema}/calculate_shipping" target="_blank">Go to Unity Catalog to see Registered Functions</a>')


# III- AI Playground 

Open the [Playground](/ml/playground) and select the tools we created to test your agent!

<div style="float: right; width: 70%;">
  <img 
    src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/\
cross_demo_assets/AI_Agent_GIFs/AI_agent_function_selection.gif" 
    alt="Function Selection" 
    width="100%"
  >
</div>

### Location Guide

Your functions are organized in Unity Catalog using this structure:

#### Example Path:
`catalog.schema.my_function`

💡 Note: Replace the example names with your actual catalog and schema names.